<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/696_MOv2_DataGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Mission Control Orchestrator v2 — Official Architecture Spec (MVP v2)

## 1) Purpose

**Mission Control** is the central coordination layer in a multi-agent system that aligns specialized agents to **measurable business missions** (e.g., onboarding time, sales velocity, support resolution).
Unlike one-off “do a task” agents, Mission Control manages an **end-to-end workflow**: decomposes the mission, routes tasks to the right agents, enforces dependencies and HITL gates, monitors progress, and measures outcomes against KPIs.

---

## 2) MVP v2 Goals

### What v2 adds (minimal data, maximum leverage)

1. **Execution history (“runs”)**: track what happened, not just what *should* happen.
2. **Task telemetry**: durations, outcomes, approvals, delays.
3. **Risk & governance signals**: detect drift from plan, SLA risk, repeated approvals/escalations.
4. **Dynamic routing**: choose agents based on lightweight performance stats (not just static capability mapping).
5. **Portfolio snapshot**: a CEO-ready view across missions.

### What v2 intentionally does NOT add (yet)

* Complex simulation, massive event streams, deep probabilistic planning, fully autonomous multi-mission scheduling, or expensive LLM-decisioning. v2 stays rule-first with optional LLM “polish.”

---

## 3) Core Inputs (v1 → reused in v2)

These remain your **source-of-truth reference data**:

### Missions

* `business_missions.json`: mission_id, name, description.

### Mission Plans (Decomposed tasks)

* `decomposed_mission_tasks.json`: ordered tasks, dependencies, estimated duration, HITL requirement.

### Agent Registry

* `specialized_agents.json`: agent_id, name, what the agent does.

### Capability Routing

* `agent_capabilities_matrix.json`: task_id → capable_agents list.

### KPI Targets

* `mission_kpis.json`: baseline + target + constraints (e.g., max steps, min touchpoints).

---

## 4) New v2 Data Model (MVP-sized)

These are the **5 new v2 datasets** (small, but unlock “operational reality”):

1. `mission_runs.json`
   **One record per execution** of a mission (start/end/status/outcome KPIs, intervention counts).

2. `task_execution_logs.json`
   **One record per task execution** within a run (agent_id used, actual duration, outcome, approval delay).

3. `mission_risk_events.json`
   **Sparse, high-signal risk events** (SLA breach risk, repeated approval loops, dependency failures).

4. `agent_performance_stats.json`
   Lightweight rolling stats per agent (avg duration, approval rate, escalation rate) used for dynamic routing.

5. `mission_portfolio_snapshot.json`
   A daily/weekly snapshot summarizing portfolio status (runs completed, missions at risk, bottlenecks).

> Design principle: **“thin telemetry”** — enough data to reason, optimize, and produce executive reporting, without becoming a data engineering project.

---

## 5) System State (Orchestrator State Object)

The Mission Control run should maintain a single state dictionary (or dataclass) with:

### Identifiers & context

* `mission_id`, `run_id`, `requested_by`, `priority`, `created_at`

### Plan & execution

* `plan_tasks[]` (from decomposed tasks)
* `task_status_by_id` (pending/running/completed/blocked/failed)
* `task_results_by_id` (outputs, errors, approvals)

### Routing & governance

* `capability_lookup` (task_id → agents)
* `agent_stats` (from v2 stats file)
* `hitl_queue[]` (approval-needed tasks)
* `risk_events[]`

### KPI tracking

* `kpi_targets` (baseline/target)
* `kpi_observations` (measured for this run)
* `kpi_gap_summary` (delta vs target)

---

## 6) Orchestration Flow (Nodes)

### Node A — Load Mission Context

**Inputs:** `mission_id`
**Loads:** mission definition , KPI targets

**Output:** mission header + KPI target block

---

### Node B — Build Mission Plan

**Loads:** decomposed tasks for mission_id (ordered + dependencies + HITL flags).

**Validations:**

* All dependencies exist
* No cycles
* Orders consistent with dependencies

**Output:** normalized execution DAG / ordered runnable queue

---

### Node C — Build Routing Lookups

**Loads:** capability matrix task→agents  + agent registry  + v2 agent stats (performance)

**Output:** `eligible_agents(task_id)` with ranked ordering

---

### Node D — Execute Next Runnable Tasks

Loop while runnable tasks exist:

1. Identify tasks whose `depends_on` are satisfied.
2. Route agent (see routing logic below)
3. If HITL required, enqueue for approval and pause execution for that task.
4. Record telemetry to `task_execution_logs.json`
5. Update state

---

### Node E — Risk & Drift Detection (v2)

Rules fire based on telemetry:

* Task duration exceeds estimate by X%
* Approval delay exceeds threshold
* Repeated rework/loop on task
* Too many escalations / failures
* Mission exceeds “time budget” implied by KPI targets

Writes sparse records to `mission_risk_events.json`.

---

### Node F — KPI Evaluation

Compute KPI observations for the run and compare to baseline/target.

Outputs:

* KPI attainment (hit/miss)
* KPI gaps
* “What to fix” ranked list (often derived from bottlenecks + HITL delays)

---

### Node G — Executive Report + Portfolio Snapshot Update

Generate:

* Run-level executive summary (“what happened, what’s at risk, what to do next”)
* Update `mission_runs.json`
* Update daily/weekly `mission_portfolio_snapshot.json`

---

## 7) Routing Logic (v2 upgrade)

v1 routing is static (“who can do task T?”) via the capabilities matrix.
v2 routing is **ranked**, using performance:

**Score(agent, task) =**

* Capability match (hard gate)
* * speed score (lower avg duration)
* * quality score (higher approval rate / lower escalation rate)
* * load score (optional later)
* * cost score (optional later)

MVP v2: implement just **capability gate + speed + escalation rate**.

---

## 8) HITL (Human-in-the-loop) Policy

HITL is already encoded per task.
v2 enforces a consistent gate:

* If `requires_human_approval = true`:

  * Task enters `hitl_queue`
  * Capture `approval_request_time`
  * Upon approval/rejection, resume or reroute
  * Log `human_delay_minutes` in `task_execution_logs.json`

---

## 9) Outputs (What the agent produces)

### Run-level outputs

* `mission_runs.json` record (status/outcomes/KPI results)
* `task_execution_logs.json` rows (telemetry)
* `mission_risk_events.json` rows (exceptions & risk signals)
* **Executive report markdown** (human-readable)

### Portfolio-level outputs

* `mission_portfolio_snapshot.json` (daily/weekly summary)
* Optional: trend charts later (not required for v2 MVP)

---

## 10) Testing Strategy (MVP v2)

1. **Plan validation tests**

   * missing dependency, cycle detection, invalid order

2. **Routing tests**

   * correct eligible agents from matrix
   * ranking changes when agent stats change

3. **HITL gate tests**

   * tasks marked HITL do not execute without approval

4. **Telemetry integrity tests**

   * every completed task generates an execution log row
   * run aggregates match task logs

5. **KPI evaluation tests**

   * baseline/target comparisons correct

---

## 11) “Why v2 matters” (Executive framing baked into the architecture)

This spec directly supports your v1 positioning: Mission Control is the “AI project manager + strategist” aligning specialized agents to measurable outcomes  and building trust via HITL safeguards .

v2 makes it **operationally real**:

* not just plans → **runs**
* not just tasks → **telemetry**
* not just output → **KPI improvement + risk signals**



# mission_runs.json

In [ ]:
[
  {
    "run_id": "RUN_2026_02_01_001",
    "mission_id": "M001",
    "start_time": "2026-02-01T14:05:00Z",
    "end_time": "2026-02-01T15:18:00Z",
    "status": "completed",
    "total_duration_minutes": 73,
    "kpis": {
      "onboarding_time_days": 3.1
    },
    "human_interventions": 2,
    "escalations": 1,
    "risk_flagged": true
  },
  {
    "run_id": "RUN_2026_02_03_002",
    "mission_id": "M001",
    "start_time": "2026-02-03T13:42:00Z",
    "end_time": "2026-02-03T14:45:00Z",
    "status": "completed",
    "total_duration_minutes": 63,
    "kpis": {
      "onboarding_time_days": 2.6
    },
    "human_interventions": 1,
    "escalations": 0,
    "risk_flagged": false
  },
  {
    "run_id": "RUN_2026_02_05_003",
    "mission_id": "M001",
    "start_time": "2026-02-05T15:02:00Z",
    "end_time": "2026-02-05T15:54:00Z",
    "status": "completed",
    "total_duration_minutes": 52,
    "kpis": {
      "onboarding_time_days": 2.1
    },
    "human_interventions": 0,
    "escalations": 0,
    "risk_flagged": false
  },

  {
    "run_id": "RUN_2026_02_02_004",
    "mission_id": "M002",
    "start_time": "2026-02-02T10:10:00Z",
    "end_time": "2026-02-02T11:01:00Z",
    "status": "completed",
    "total_duration_minutes": 51,
    "kpis": {
      "pipeline_days": 10
    },
    "human_interventions": 1,
    "escalations": 0,
    "risk_flagged": false
  },
  {
    "run_id": "RUN_2026_02_04_005",
    "mission_id": "M002",
    "start_time": "2026-02-04T09:25:00Z",
    "end_time": "2026-02-04T10:42:00Z",
    "status": "completed",
    "total_duration_minutes": 77,
    "kpis": {
      "pipeline_days": 8
    },
    "human_interventions": 2,
    "escalations": 1,
    "risk_flagged": true
  },

  {
    "run_id": "RUN_2026_02_01_006",
    "mission_id": "M003",
    "start_time": "2026-02-01T16:30:00Z",
    "end_time": "2026-02-01T16:57:00Z",
    "status": "completed",
    "total_duration_minutes": 27,
    "kpis": {
      "resolution_time_hours": 5.2,
      "csat": 3.8
    },
    "human_interventions": 1,
    "escalations": 1,
    "risk_flagged": true
  },
  {
    "run_id": "RUN_2026_02_03_007",
    "mission_id": "M003",
    "start_time": "2026-02-03T17:05:00Z",
    "end_time": "2026-02-03T17:26:00Z",
    "status": "completed",
    "total_duration_minutes": 21,
    "kpis": {
      "resolution_time_hours": 3.9,
      "csat": 4.3
    },
    "human_interventions": 0,
    "escalations": 0,
    "risk_flagged": false
  },
  {
    "run_id": "RUN_2026_02_06_008",
    "mission_id": "M003",
    "start_time": "2026-02-06T11:18:00Z",
    "end_time": "2026-02-06T11:35:00Z",
    "status": "completed",
    "total_duration_minutes": 17,
    "kpis": {
      "resolution_time_hours": 3.2,
      "csat": 4.6
    },
    "human_interventions": 0,
    "escalations": 0,
    "risk_flagged": false
  }
]


# task_execution_logs.json

In [ ]:
[
  {
    "run_id": "RUN_2026_02_01_001",
    "mission_id": "M001",
    "task_id": "T1",
    "agent_id": "A1",
    "actual_duration_minutes": 6,
    "human_approval_required": false,
    "human_delay_minutes": 0,
    "outcome": "completed"
  },
  {
    "run_id": "RUN_2026_02_01_001",
    "mission_id": "M001",
    "task_id": "T2",
    "agent_id": "A2",
    "actual_duration_minutes": 16,
    "human_approval_required": true,
    "human_delay_minutes": 7,
    "outcome": "approved"
  },
  {
    "run_id": "RUN_2026_02_01_001",
    "mission_id": "M001",
    "task_id": "T3",
    "agent_id": "A3",
    "actual_duration_minutes": 4,
    "human_approval_required": false,
    "human_delay_minutes": 0,
    "outcome": "completed"
  },

  {
    "run_id": "RUN_2026_02_03_002",
    "mission_id": "M001",
    "task_id": "T1",
    "agent_id": "A1",
    "actual_duration_minutes": 5,
    "human_approval_required": false,
    "human_delay_minutes": 0,
    "outcome": "completed"
  },
  {
    "run_id": "RUN_2026_02_03_002",
    "mission_id": "M001",
    "task_id": "T2",
    "agent_id": "A2",
    "actual_duration_minutes": 13,
    "human_approval_required": true,
    "human_delay_minutes": 4,
    "outcome": "approved"
  },
  {
    "run_id": "RUN_2026_02_03_002",
    "mission_id": "M001",
    "task_id": "T3",
    "agent_id": "A3",
    "actual_duration_minutes": 3,
    "human_approval_required": false,
    "human_delay_minutes": 0,
    "outcome": "completed"
  },

  {
    "run_id": "RUN_2026_02_05_003",
    "mission_id": "M001",
    "task_id": "T1",
    "agent_id": "A1",
    "actual_duration_minutes": 4,
    "human_approval_required": false,
    "human_delay_minutes": 0,
    "outcome": "completed"
  },
  {
    "run_id": "RUN_2026_02_05_003",
    "mission_id": "M001",
    "task_id": "T2",
    "agent_id": "A2",
    "actual_duration_minutes": 11,
    "human_approval_required": true,
    "human_delay_minutes": 2,
    "outcome": "approved"
  },
  {
    "run_id": "RUN_2026_02_05_003",
    "mission_id": "M001",
    "task_id": "T3",
    "agent_id": "A3",
    "actual_duration_minutes": 3,
    "human_approval_required": false,
    "human_delay_minutes": 0,
    "outcome": "completed"
  },

  {
    "run_id": "RUN_2026_02_02_004",
    "mission_id": "M002",
    "task_id": "T4",
    "agent_id": "A4",
    "actual_duration_minutes": 9,
    "human_approval_required": false,
    "human_delay_minutes": 0,
    "outcome": "completed"
  },
  {
    "run_id": "RUN_2026_02_02_004",
    "mission_id": "M002",
    "task_id": "T5",
    "agent_id": "A4",
    "actual_duration_minutes": 7,
    "human_approval_required": true,
    "human_delay_minutes": 3,
    "outcome": "approved"
  },

  {
    "run_id": "RUN_2026_02_04_005",
    "mission_id": "M002",
    "task_id": "T4",
    "agent_id": "A4",
    "actual_duration_minutes": 12,
    "human_approval_required": false,
    "human_delay_minutes": 0,
    "outcome": "completed"
  },
  {
    "run_id": "RUN_2026_02_04_005",
    "mission_id": "M002",
    "task_id": "T5",
    "agent_id": "A4",
    "actual_duration_minutes": 9,
    "human_approval_required": true,
    "human_delay_minutes": 6,
    "outcome": "approved"
  },

  {
    "run_id": "RUN_2026_02_01_006",
    "mission_id": "M003",
    "task_id": "T6",
    "agent_id": "A5",
    "actual_duration_minutes": 3,
    "human_approval_required": false,
    "human_delay_minutes": 0,
    "outcome": "completed"
  },
  {
    "run_id": "RUN_2026_02_01_006",
    "mission_id": "M003",
    "task_id": "T7",
    "agent_id": "A5",
    "actual_duration_minutes": 5,
    "human_approval_required": false,
    "human_delay_minutes": 0,
    "outcome": "completed"
  },
  {
    "run_id": "RUN_2026_02_01_006",
    "mission_id": "M003",
    "task_id": "T8",
    "agent_id": "A5",
    "actual_duration_minutes": 6,
    "human_approval_required": true,
    "human_delay_minutes": 5,
    "outcome": "approved"
  },

  {
    "run_id": "RUN_2026_02_03_007",
    "mission_id": "M003",
    "task_id": "T6",
    "agent_id": "A5",
    "actual_duration_minutes": 2,
    "human_approval_required": false,
    "human_delay_minutes": 0,
    "outcome": "completed"
  },
  {
    "run_id": "RUN_2026_02_03_007",
    "mission_id": "M003",
    "task_id": "T7",
    "agent_id": "A5",
    "actual_duration_minutes": 4,
    "human_approval_required": false,
    "human_delay_minutes": 0,
    "outcome": "completed"
  },
  {
    "run_id": "RUN_2026_02_03_007",
    "mission_id": "M003",
    "task_id": "T8",
    "agent_id": "A5",
    "actual_duration_minutes": 5,
    "human_approval_required": true,
    "human_delay_minutes": 2,
    "outcome": "approved"
  },

  {
    "run_id": "RUN_2026_02_06_008",
    "mission_id": "M003",
    "task_id": "T6",
    "agent_id": "A5",
    "actual_duration_minutes": 2,
    "human_approval_required": false,
    "human_delay_minutes": 0,
    "outcome": "completed"
  },
  {
    "run_id": "RUN_2026_02_06_008",
    "mission_id": "M003",
    "task_id": "T7",
    "agent_id": "A5",
    "actual_duration_minutes": 3,
    "human_approval_required": false,
    "human_delay_minutes": 0,
    "outcome": "completed"
  },
  {
    "run_id": "RUN_2026_02_06_008",
    "mission_id": "M003",
    "task_id": "T8",
    "agent_id": "A5",
    "actual_duration_minutes": 4,
    "human_approval_required": true,
    "human_delay_minutes": 1,
    "outcome": "approved"
  }
]


# mission_risk_events.json

In [ ]:
[
  {
    "event_id": "RISK_001",
    "run_id": "RUN_2026_02_01_001",
    "mission_id": "M001",
    "task_id": "T2",
    "risk_type": "sla_breach_risk",
    "severity": "medium",
    "description": "Document verification exceeded estimated duration and required extended human review.",
    "detected_at": "2026-02-01T14:40:00Z",
    "status": "closed"
  },
  {
    "event_id": "RISK_002",
    "run_id": "RUN_2026_02_04_005",
    "mission_id": "M002",
    "task_id": "T5",
    "risk_type": "approval_delay",
    "severity": "high",
    "description": "Personalized outreach email approval exceeded time threshold.",
    "detected_at": "2026-02-04T10:15:00Z",
    "status": "closed"
  },
  {
    "event_id": "RISK_003",
    "run_id": "RUN_2026_02_01_006",
    "mission_id": "M003",
    "task_id": "T8",
    "risk_type": "csat_risk",
    "severity": "high",
    "description": "Resolution message approval delay correlated with CSAT below threshold.",
    "detected_at": "2026-02-01T16:52:00Z",
    "status": "closed"
  },
  {
    "event_id": "RISK_004",
    "run_id": "RUN_2026_02_01_006",
    "mission_id": "M003",
    "task_id": "T8",
    "risk_type": "resolution_time_breach",
    "severity": "medium",
    "description": "Support resolution exceeded target time window.",
    "detected_at": "2026-02-01T16:55:00Z",
    "status": "closed"
  },
  {
    "event_id": "RISK_005",
    "run_id": "RUN_2026_02_04_005",
    "mission_id": "M002",
    "task_id": "T4",
    "risk_type": "task_overrun",
    "severity": "low",
    "description": "Deal analysis task exceeded estimated duration.",
    "detected_at": "2026-02-04T09:48:00Z",
    "status": "closed"
  }
]


# agent_performance_stats.json

In [ ]:
[
  {
    "agent_id": "A1",
    "avg_task_duration_minutes": 5.0,
    "approval_rate": 1.0,
    "escalation_rate": 0.0,
    "last_updated": "2026-02-06"
  },
  {
    "agent_id": "A2",
    "avg_task_duration_minutes": 13.3,
    "approval_rate": 0.95,
    "escalation_rate": 0.10,
    "last_updated": "2026-02-06"
  },
  {
    "agent_id": "A3",
    "avg_task_duration_minutes": 3.3,
    "approval_rate": 1.0,
    "escalation_rate": 0.0,
    "last_updated": "2026-02-06"
  },
  {
    "agent_id": "A4",
    "avg_task_duration_minutes": 9.3,
    "approval_rate": 0.90,
    "escalation_rate": 0.15,
    "last_updated": "2026-02-06"
  },
  {
    "agent_id": "A5",
    "avg_task_duration_minutes": 4.2,
    "approval_rate": 0.93,
    "escalation_rate": 0.08,
    "last_updated": "2026-02-06"
  }
]


# mission_portfolio_snapshot.json

In [ ]:
[
  {
    "snapshot_date": "2026-02-02",
    "active_runs": 3,
    "completed_runs": 2,
    "missions_at_risk": 1,
    "avg_kpi_improvement_pct": 18,
    "top_bottleneck_task_id": "T2",
    "top_bottleneck_task_name": "Verify documents",
    "escalations_last_24h": 2,
    "human_interventions_last_24h": 3
  },
  {
    "snapshot_date": "2026-02-04",
    "active_runs": 2,
    "completed_runs": 5,
    "missions_at_risk": 2,
    "avg_kpi_improvement_pct": 24,
    "top_bottleneck_task_id": "T5",
    "top_bottleneck_task_name": "Draft personalized outreach email",
    "escalations_last_24h": 2,
    "human_interventions_last_24h": 4
  },
  {
    "snapshot_date": "2026-02-06",
    "active_runs": 1,
    "completed_runs": 8,
    "missions_at_risk": 0,
    "avg_kpi_improvement_pct": 33,
    "top_bottleneck_task_id": "T8",
    "top_bottleneck_task_name": "Draft resolution message",
    "escalations_last_24h": 0,
    "human_interventions_last_24h": 1
  }
]
